## Second Assignment
This notebook is for the 3 items in the second assignment for Week 3 in the Applied Data Science Capstone course.

### Item #1 Scraping the Wiki page for data linking the postal codes to Boroughs and Neighborhoods in Toronto

Simply using pandas package to read tables from this Wiki page https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M. 

In [1]:
import pandas as pd
import numpy as np

dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M') 
print('There are {} tables.'.format(len(dfs)))

There are 3 tables.


The first table is what is needed.

In [2]:
print(dfs[0].shape)
dfs[0].head()

(180, 3)


,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Drop rows with Borough=='Not assigned' and assign Borough information to Neighborhood if the latter is "Not assigned".

In [3]:
# assign dfs[0] to a new dataframe
df_Toronto = dfs[0]

# drop rows with Borough=='Not assigned'
df_Toronto = df_Toronto[df_Toronto.Borough != 'Not assigned'].reset_index(drop=True)
print('The shape of the data',df_Toronto.shape)
df_Toronto.head()

The shape of the data (103, 3)


,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Checking if any "Not assigned" Neighborhood exist after dropping "Not assigned" Boroughs.

In [4]:
(df_Toronto.Neighborhood=='Not assigned').value_counts() # 103 False, no "Not assigned" Neighborhood after dropping "Not assigned" Boroughs
#(df_Toronto.Neighborhood==df_Toronto.Borough).value_counts()

# modify the Neighborhood value from "Not assigned" to the corresponding Borough value, did not run since none such row
#df_Toronto['Neighborhood']= np.where(df_Toronto['Neighborhood'] =='Not assigned', df_Toronto['Borough'], df_Toronto['Neighborhood'])


False    103
Name: Neighborhood, dtype: int64

The code below renames the first column to "PostalCode" from "Postal Code". But decided not to do this because the provided geo coordinate file uses "Postal Code" as well.

In [5]:
# skip renaming "Postal Code" to "PostalCode", since the former matches the column name in the coordinate file provided
#df_Toronto.rename(columns = {'Postal Code':'PostalCode'},inplace=True) # or use df_Toronto.columns.values[0]='PostalCode'? No, the latter make it not indexible by column name
#df_Toronto.head()

In [6]:
print('There are {} rows of data.'.format(df_Toronto.shape[0]))

There are 103 rows of data.


### Item #2 Adding the latitude and longitude coordinates to the df_Toronto dataframe

Tried the Geocoder package. After waiting for a long time, decided to terminate the process and instead use the provided csv file for the geo coordinate data. Then merged it with df_Toronto.


In [24]:
!conda install -c conda-forge geocoder --yes # Installed successfully, commented out and re-run to clear the output

In [23]:
# this code chunk runs well, but it is taking way too long to finish, quit and decide to load the provided csv file instead

import geocoder # import geocoder  

df_coord = pd.DataFrame(columns=['PostalCode','Latitude', 'Longitude'])
for pCode in df_Toronto['PostalCode']:
    # initialize your variable to None
    lat_lng_coords = None
    
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(pCode))
        lat_lng_coords = g.latlng
        
    df_coord = df_coord.append({'PostalCode': pCode,
                                          'Latitude': lat_lng_coords[0],
                                          'Longitude': lat_lng_coords[1]}, ignore_index=True)
    
df_coord.head()

In [10]:
CoordFromFile = pd.read_csv('http://cocl.us/Geospatial_data')
CoordFromFile.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
# merge/add latitude and longitude to df_Toronto
df_Toronto=pd.merge(df_Toronto,CoordFromFile)

df_Toronto.head()


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [13]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_Toronto['Borough'].unique()),
        df_Toronto.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [89]:
#df_Toronto.groupby('Borough').count() # a bit more info on each borough

### Item #3 Explore Toronto using K-means clustering

First, load the libraries needed.

In [48]:
### libraries needed for pulling data from Fouesquare, as well as downstream processing

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # commented out for rerun to get rid of the output
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe, deprecated, use pandas.json_normalize
#from pandas import json_normalize

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # commented out for rerun to get rid of the output
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### Use geopy library to get the latitude and longitude values of Toronto.
Setup <em>toronto_explorer</em> as the user_agent to define an instance of the geocoder.

In [16]:
# get the lat and long coordinate for Toronto
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Use map to visualize the neighborhoods around Toronto

In [17]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers per postal code locations to map
for lat, lng, borough, neighborhood in zip(df_Toronto['Latitude'], df_Toronto['Longitude'], df_Toronto['Borough'], df_Toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

### **_NOTE_**, after commit the file to Github, the map doesn't show up. Would it be ok to submit a link via Watson Studio sharing?

#### Narrow down to those neighborhoods in boroughs containing "Toronto" in the name.

In [23]:
#filter down to boroughs containing "Toronto"
#toronto_data = df_Toronto[df_Toronto['Borough'].isin(['Central Toronto','Downtown Toronto','East Toronto','West Toronto'])].reset_index(drop=True) #this way or below
toronto_data = df_Toronto[df_Toronto['Borough'].str.contains('Toronto')].reset_index(drop=True)
print(toronto_data.shape)
toronto_data.head()

(39, 5)


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


#### Visualize this subset of neighborhoods around Toronto.

In [24]:
# create map of Toronto with boroughs containing "Toronto", again using latitude and longitude values obtained above
map_torontoB = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_torontoB)  
    
map_torontoB

### **_NOTE_**, after commit the file to Github, the map doesn't show up. Would it be ok to submit a link via Watson Studio sharing?

#### Obtain venues data for each neighborhoods in the narrowed down subset using Foursquare API
First, setting up the required common request parameters. Hiding the credential info after running the code below.

In [51]:
# The code was removed by Watson Studio for sharing.

#### Testing out on one neighborhood to ensure the data extraction process works well.

In [26]:
## checking the first neighborhood in the dataframe to make sure the process works first
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

LIMIT = 100 # number of venues to return
radius = 500 # in meters
url='https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)

url

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


'https://api.foursquare.com/v2/venues/explore?client_id=050Q4CT3UEKITFTDAS55I5X4IA4FQPQWAYOGZ0DWOUAAIBN1&client_secret=RVWE241BEXCO51ESEWYQDMFDR1JKC0W5TW3HVFQSVQIH1QS0&ll=43.6542599,-79.3606359&v=20180605&radius=500&limit=100'

In [27]:
results = requests.get(url).json()
#results

From the course labs, we know that all the information is in the *items* key. We could borrow the **get_category_type** function to extract category of the venue once we extract the *items* from the *results* returned from the requests.

In [28]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [90]:
# extract and clean the desired data returned from Foursquare, for one neighborhood as a test
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter/extract the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

#nearby_venues.head()

#### Borrow the function **getNearbyVenues** to repeat the same process above to all the neighborhoods in Boroughs containing "Toronto".

In [91]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [92]:
# get the venues data for all the neighborhoods in the narrowed down subset
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

In [93]:
# checking the cleaned up data extracted fron Foursquare
print(toronto_venues.shape)
#toronto_venues.head()


(1634, 7)


In [94]:
#learn a bit more about the 

print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))
#toronto_venues.groupby('Neighborhood').count()

There are 234 unique categories.


#### Prep the data for clustering using K-means
Using one hot coding to create dummy columns for each venue category, then calculate the frequency of each venue category within a neighborhood. This is the data (minus the Neighborhood column) for K-means clustering.

In [95]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
#toronto_grouped

Again, borrow the function **return_most_common_venues** from labs to sort the venues in descending order within each neighborhood. Then putting this data into a pandas dataframe. This is will be merged with the K-means output labels for examining the clusters later.

In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues] # could add a condition to replace the index.value with "NA" if the value is 0.0

In [97]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

#neighborhoods_venues_sorted.head()

#### Perform K-means clustering analysis to segment the neighborhoods into 5 clusters.
Could try other number to segment into more or less clusters.

In [61]:
# set number of clusters
kclusters = 5 # tried 4, one group seems too big

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 3, 0, 0, 3, 0, 0, 3, 3], dtype=int32)

Incorporate the clusering labels into the data with top 10 most common venues data and then merge back with the narrowed down neighborhood subset.

In [98]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Breakfast Spot,Café,Pub,Theater,Mexican Restaurant,Shoe Store,Restaurant
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,College Cafeteria,Diner,Creperie,Park,Music Venue,Mexican Restaurant,Italian Restaurant,Hobby Shop,Fried Chicken Joint
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Japanese Restaurant,Italian Restaurant,Cosmetics Shop,Café,Bubble Tea Shop,Hotel,Bookstore,Fast Food Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Restaurant,Café,Clothing Store,Cocktail Bar,American Restaurant,Cosmetics Shop,Creperie,Breakfast Spot,Moroccan Restaurant
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,3,Trail,Health Food Store,Pub,Women's Store,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Dumpling Restaurant


#### Visualize the clusters of neighborhoods on the Toronto map

In [63]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
colors_array = cm.rainbow(np.linspace(0, 1, kclusters))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1], #last color to the first cluster
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### **_NOTE_**, after commit the file to Github, the map doesn't show up. Would it be ok to submit a link via Watson Studio sharing?

#### Examine each cluster group for insights

Summary how many neighborhoods in each cluster and show details of what venues are most commonly found in each neighborhood by cluster group.

In [83]:
toronto_merged['Cluster Labels'].value_counts()

3    18
0    16
1     3
4     1
2     1
Name: Cluster Labels, dtype: int64

Cluster 3 has the most neighborhoods, 18; cluster 0 has 16 neighborhoods, second most; cluster 1 has 3 neighborhoods; while cluster 2 and 4 each only have one neighborhood. 

In [43]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Park,Bakery,Breakfast Spot,Café,Pub,Theater,Mexican Restaurant,Shoe Store,Restaurant
1,Downtown Toronto,0,Coffee Shop,College Cafeteria,Diner,Creperie,Park,Music Venue,Mexican Restaurant,Italian Restaurant,Hobby Shop,Fried Chicken Joint
2,Downtown Toronto,0,Clothing Store,Coffee Shop,Japanese Restaurant,Italian Restaurant,Cosmetics Shop,Café,Bubble Tea Shop,Hotel,Bookstore,Fast Food Restaurant
3,Downtown Toronto,0,Coffee Shop,Restaurant,Café,Clothing Store,Cocktail Bar,American Restaurant,Cosmetics Shop,Creperie,Breakfast Spot,Moroccan Restaurant
5,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Bakery,Pharmacy,Cheese Shop,Café,Restaurant,Farmers Market,Beer Bar,Seafood Restaurant
6,Downtown Toronto,0,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Japanese Restaurant,Thai Restaurant,Salad Place,Burger Joint,Bar,Department Store
8,Downtown Toronto,0,Coffee Shop,Café,Clothing Store,Restaurant,Hotel,Deli / Bodega,Thai Restaurant,Gym,Bookstore,Pizza Place
10,Downtown Toronto,0,Coffee Shop,Aquarium,Hotel,Café,Sporting Goods Shop,Fried Chicken Joint,Brewery,Restaurant,Scenic Lookout,Bakery
11,West Toronto,0,Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Café,Restaurant,Men's Store,Asian Restaurant,Yoga Studio,Dog Run
13,Downtown Toronto,0,Coffee Shop,Hotel,Café,Restaurant,Salad Place,Japanese Restaurant,Italian Restaurant,Seafood Restaurant,American Restaurant,Sushi Restaurant


In [44]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,1,Park,Bus Line,Swim School,Farmers Market,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
21,Central Toronto,1,Park,Trail,Jewelry Store,Sushi Restaurant,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
33,Downtown Toronto,1,Park,Playground,Trail,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [65]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1,2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,Roselawn,2,Garden,Women's Store,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [46]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,East Toronto,3,Trail,Health Food Store,Pub,Women's Store,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Dumpling Restaurant
7,Downtown Toronto,3,Grocery Store,Café,Park,Coffee Shop,Italian Restaurant,Candy Store,Diner,Restaurant,Baby Store,Nightclub
9,West Toronto,3,Bakery,Pharmacy,Café,Recording Studio,Middle Eastern Restaurant,Bar,Supermarket,Bank,Pizza Place,Brewery
12,East Toronto,3,Greek Restaurant,Italian Restaurant,Coffee Shop,Restaurant,Ice Cream Shop,Furniture / Home Store,Yoga Studio,Liquor Store,Spa,Bookstore
14,West Toronto,3,Café,Coffee Shop,Breakfast Spot,Gym,Stadium,Burrito Place,Restaurant,Climbing Gym,Pet Store,Bakery
15,East Toronto,3,Park,Sandwich Place,Pizza Place,Sushi Restaurant,Brewery,Burrito Place,Restaurant,Pub,Pet Store,Coffee Shop
17,East Toronto,3,Café,Coffee Shop,Brewery,Gastropub,American Restaurant,Bakery,Music Store,Seafood Restaurant,Sandwich Place,Cheese Shop
20,Central Toronto,3,Sandwich Place,Park,Department Store,Gym / Fitness Center,Pizza Place,Breakfast Spot,Dance Studio,Food & Drink Shop,Hotel,Women's Store
22,West Toronto,3,Mexican Restaurant,Café,Thai Restaurant,Bookstore,Fried Chicken Joint,Park,Music Venue,Diner,Discount Store,Cajun / Creole Restaurant
24,Central Toronto,3,Café,Sandwich Place,Coffee Shop,Pizza Place,Indian Restaurant,Liquor Store,BBQ Joint,History Museum,Middle Eastern Restaurant,Pub


In [64]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1,2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,Central Toronto,"Moore Park, Summerhill East",4,Playground,Tennis Court,Summer Camp,Women's Store,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


#### Some extra ideas to try in the future

Maybe the number of venues found could be used as a feature too?

In [82]:
venuesCnt = toronto_onehot.groupby('Neighborhood').count()[['Airport']].reset_index()
venuesCnt.rename(columns = {'Airport':'VenuesCnt'},inplace=True)
venuesCnt.head()

,Neighborhood,VenuesCnt
0,Berczy Park,57
1,"Brockton, Parkdale Village, Exhibition Place",22
2,"Business reply mail Processing Centre, South C...",16
3,"CN Tower, King and Spadina, Railway Lands, Har...",16
4,Central Bay Street,66


Modify the **return_most_common_venues** to leave blank if venue categories freqency is 0.0. This will be more useful when examining the pattern of the neighborhoods in each cluster. The current data is misleading in filling in 0 freqency venue categories as the nth most common venues if less than 10 categories are found in a neighborhood.

In [86]:
def return_most_common_venuesM(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    cateories_sorted = row_categories_sorted.index.values #[0:num_top_venues]
    mask =  row_categories_sorted == 0.0
    cateories_sorted[mask] = ''
    
    return cateories_sorted[0:num_top_venues] # could add a condition to replace the index.value with "NA" if the value is 0.0

In [99]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sortedM = pd.DataFrame(columns=columns)
neighborhoods_venues_sortedM['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sortedM.iloc[ind, 1:] = return_most_common_venuesM(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sortedM.tail()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
34,"The Annex, North Midtown, Yorkville",Café,Sandwich Place,Coffee Shop,Pizza Place,Indian Restaurant,Liquor Store,BBQ Joint,History Museum,Middle Eastern Restaurant,Pub
35,The Beaches,Trail,Health Food Store,Pub,,,,,,,
36,"The Danforth West, Riverdale",Greek Restaurant,Italian Restaurant,Coffee Shop,Restaurant,Ice Cream Shop,Furniture / Home Store,Yoga Studio,Liquor Store,Spa,Bookstore
37,"Toronto Dominion Centre, Design Exchange",Coffee Shop,Hotel,Café,Restaurant,Salad Place,Japanese Restaurant,Italian Restaurant,Seafood Restaurant,American Restaurant,Sushi Restaurant
38,"University of Toronto, Harbord",Café,Bar,Japanese Restaurant,Bookstore,Sandwich Place,Restaurant,Bakery,Yoga Studio,Pub,Beer Bar
